# Imports

In [1]:
# sklearn packages
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFECV

# Statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle

# utility libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 200)
sns.set(style="whitegrid")

%matplotlib inline

# Functions

In [19]:
def make_polynomial(df_): # last step before modelling
    poly_ = PolynomialFeatures(degree=2, include_bias=False)
    poly_data = poly_.fit_transform(df_)
    poly_columns = poly_.get_feature_names(df_.columns)
    
    return pd.DataFrame(poly_data, columns=poly_columns)
"""Don't poly dummies (1**2 = 1)"""

"Don't poly dummies (1**2 = 1)"

In [3]:
def run_lin_reg_model(df, target, random_state=34, test_size=0.2):
    
    X_train_, X_test_, y_train_, y_test_ = train_test_split(
        df, target, random_state=34, test_size=0.2) # <---train test split
    
    lm = linear_model.LinearRegression().fit(
        X_train_, y_train_) # <---linear regression model
    
    train_rmse = np.sqrt(metrics.mean_squared_error(
        y_train_, lm.predict(X_train_))) # <---training RMSE
    
    test_rmse = np.sqrt(metrics.mean_squared_error(
        y_test_, lm.predict(X_test_))) # <---test RMSE
    
#     print(list(zip(df.columns, lm.coef_))) #<---check coeffiecients
    
    return X_train_, X_test_, y_train_, y_test_, lm, train_rmse, test_rmse

In [4]:
def remove_correlations(X_train_, X_test_):
    corr_matrix = X_train_.corr().abs() # <---create correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)) # <---select upper triangle of correlation matrix
    to_drop = [column for column in upper.columns if any(upper[column] > 0.90)] # <---find index of feature columns with correlation greater than 0.90

    X_train_ = X_train_.drop(columns=to_drop, inplace=False)
    X_test_ = X_test_.drop(columns=to_drop, inplace=False)
    
    return X_train_, X_test_

In [5]:
def get_vif(X_train_): #run before poly to get rid of high correlation features
    vif_ = pd.DataFrame()
    vif_["VIF Factor"] = [variance_inflation_factor(X_train_.values, i) for i in range(X_train_.shape[1])]
    vif_["features"] = X_train_.columns
    return vif_.round(1)

In [6]:
"""how does each individual feature perform relative to the target

looks at how each feature performs relative to the others

run k-best a second time using entire data set to get coefficients for final model


run k_best to reduce correlated catagories before lasso"""

def get_k_best(X_train_, X_test_, y_train_, y_test_):
    selector = SelectKBest(f_regression, k=50) # <--- F-Test (filter method; before fitting model)
    selector.fit(X_train_, y_train_)
    selected_columns = X_train_.columns[selector.get_support()]
#     removed_columns = X_train_.columns[~selector.get_support()]

    X_train_ = X_train_[selected_columns]
    X_test_ = X_test_[selected_columns]
    
    lm = linear_model.LinearRegression().fit(
        X_train_[selected_columns], y_train_) # <---linear regression model

    train_rmse = np.sqrt(metrics.mean_squared_error(
        y_train_, lm.predict(X_train_[selected_columns]))) # <---training RMSE

    test_rmse = np.sqrt(metrics.mean_squared_error(
        y_test_, lm.predict(X_test_[selected_columns]))) # <---test RMSE

    return X_train_, X_test_, lm, train_rmse, test_rmse

In [7]:
def get_p_values():
    """run model then check p-values and remove highest as a manual wrapper method 
    or to explain rfe (auto wrapper method)"""

In [8]:
"""cross validation breaks training set into smaller train/tests and takes the average to
hedge against variance

wrapper method can check if accounting for interactions/variation in other features makes
a feature more important than just an f-test would show, puts in context of whole model

verbose for feedback"""

def get_rfe(X_train_, X_test_, y_train_, y_test_):
    ols = linear_model.LinearRegression() # <---create recursive feature eliminator
    selector = RFECV(estimator=ols,
                     step=1,
                     cv=5,
                     scoring='neg_mean_squared_error',
                     n_jobs=-1) # <---score features by mean squared errors (increase step size to make faster)
    selector.fit(X_train_, y_train_) #<---fit recursive feature eliminator     
    selected_columns = X_train_.columns[selector.support_]
#     removed_columns = X_train_.columns[~selector.support_]
    
    X_train_ = X_train_[selected_columns]
    X_test_ = X_test_[selected_columns]
    
    lm = linear_model.LinearRegression().fit(
        X_train_[selected_columns], y_train_) # <---linear regression model

    train_rfe_rmse = np.sqrt(metrics.mean_squared_error(
        y_train_, lm.predict(X_train_[selected_columns]))) # <---training RMSE

    test_rfe_rmse = np.sqrt(metrics.mean_squared_error(
        y_test_, lm.predict(X_test_[selected_columns]))) # <---test RMSE

    return X_train_, X_test_, lm, train_rmse, test_rmse

In [9]:
"""can you tell from the test rmse and train rmse that your model is not overfitting?
(test rmse is good relative to train rmse)


investigate what the selected columns are before passing to holdout"""

def finalize(df, target):
    """run kbest on ENTiRE dataset"""
    selector = SelectKBest(f_regression, k=20)
    selector.fit(df, target)

    selected_columns = df.columns[selector.get_support()]
    removed_columns = df.columns[~selector.get_support()]

    lm = linear_model.LinearRegression().fit(df[selected_columns], target) # <---fit the linear regression to the data
    lm.coef_ # <---final model for entire dataset

    pickle_out = open("model.pickle","wb")
    pickle.dump(lm, pickle_out)
    pickle_out.close()

    pickle_out = open("selected_columns.pickle","wb")
    pickle.dump(selected_columns, pickle_out)
    pickle_out.close()

    # pickle_out = open("scaler.pickle", "wb")
    # pickle.dump(scaler, pickle_out)
    # pickle_out.close

# Main

In [10]:
# import raw data
df = pd.read_csv('clean_data_final.csv', index_col=0)

In [11]:
target = df['price']

In [12]:
df.drop(columns='price', inplace=True)

In [13]:
X_train, X_test, y_train, y_test, lm, train_rmse, test_rmse = run_lin_reg_model(df, target)

In [14]:
train_rmse

163795.2336650525

In [15]:
test_rmse

170398.1073419808

In [20]:
df_poly = make_polynomial(df)

In [21]:
X_train_poly, X_test_poly, y_train_poly, y_test_poly, lm_poly, train_rmse_poly, test_rmse_poly = run_lin_reg_model(df, target)

In [22]:
train_rmse_poly, test_rmse_poly

(163795.2336650525, 170398.1073419808)

In [ ]:
X_train, X_test = remove_correlations(X_train, X_test)

In [ ]:
train_rmse, test_rmse

In [ ]:
vif = get_vif(X_train)

In [ ]:
X_train_kb, X_test_kb, lm_kb, train_rmse_kb, test_rmse_kb = get_k_best(X_train, X_test, y_train, y_test)

In [ ]:
train_rmse_kb, test_rmse_kb

In [23]:
X_train_rfe, X_test_rfe, lm_rfe, train_rmse_rfe, test_rmse_rfe = get_rfe(X_train, X_test, y_train, y_test)

In [24]:
train_rmse_rfe, test_rmse_rfe

(163795.2336650525, 170398.1073419808)

In [ ]:
finalize(df, target)

In [ ]:
test_rmse


In [ ]:
df.shape

In [25]:
train_rmse_rfe, test_rmse_rfe

(163795.2336650525, 170398.1073419808)